In [ ]:
# Notebook: PPO on Random-Start LunarLander

# Cell 1: Training on random starts

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat
from stable_baselines3.common.utils import get_linear_fn
import pandas as pd
import time
from statistics import mean, stdev

# 1) Wrapper: randomize the lander’s initial x (pad remains at x=0)
class RandomStartWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        shift = self.np_random.uniform(-0.4, 0.4)
        uw = self.unwrapped
        # teleport lander body
        uw.lander.position = (uw.lander.position.x + shift, uw.lander.position.y)
        # adjust obs so obs[0] reflects new x
        obs = obs.copy()
        obs[0] += shift
        return obs, info
    def step(self, action):
        return super().step(action)

# 2) Wrapper: track fuel & landing error
class LunarWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        return obs, info
    def step(self, action):
        obs, reward, term, trunc, info = super().step(action)
        info["fuel_used"]   = 1.0 if action==2 else 0.0
        info["landing_err"] = np.linalg.norm(obs[0:2])
        return obs, reward, term, trunc, info

# 3) Callback: log custom metrics to TensorBoard
class MetricsCallback(BaseCallback):
    def _on_training_start(self):
        if not any(isinstance(fmt, TensorBoardOutputFormat)
                   for fmt in self.logger.output_formats):
            self.logger.configure(format_strings=["tensorboard"])
    def _on_step(self) -> bool:
        for info in self.locals["infos"]:
            if "episode" in info:
                self.logger.record("custom/fuel_used",   info["fuel_used"])
                self.logger.record("custom/landing_err", info["landing_err"])
        return True

# 4) Build VecEnv
def make_env():
    e = gym.make("LunarLander-v3")
    e = RandomStartWrapper(e)
    e = LunarWrapper(e)
    return e

env = DummyVecEnv([make_env])
env = VecMonitor(env)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

# 5) Instantiate & train PPO
model = PPO(
    policy="MlpPolicy",
    env=env,
    tensorboard_log="./ppo_lunar_tb",
    verbose=1,
    n_steps=4096,
    batch_size=256,
    learning_rate=get_linear_fn(5e-4,1e-5,0.1),
    clip_range=0.15,
    ent_coef=0.005,
    policy_kwargs=dict(net_arch=[dict(pi=[128,128], vf=[128,128])]),
)
model.learn(total_timesteps=1_000_000, callback=MetricsCallback(), tb_log_name="randomstart_train")
model.save("ppo_lunar_randstart")
env.save("ppo_vecnormalize.pkl")
print("Training on random-start complete")


<frozen importlib._bootstrap>:228: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
c:\Users\a13hk\anaconda3\envs\ll_env\lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Using cpu device
Logging to ./ppo_lunar_tb\randomstart_train_5
-----------------------------------
| custom/            |            |
|    fuel_used       | 0          |
|    landing_err     | 0.14702405 |
| rollout/           |            |
|    ep_len_mean     | 89.5       |
|    ep_rew_mean     | -183       |
| time/              |            |
|    fps             | 1334       |
|    iterations      | 1          |
|    time_elapsed    | 3          |
|    total_timesteps | 4096       |
-----------------------------------
-----------------------------------------
| custom/                 |             |
|    fuel_used            | 0           |
|    landing_err          | 1.0307155   |
| rollout/                |             |
|    ep_len_mean          | 90.5        |
|    ep_rew_mean          | -167        |
| time/                   |             |
|    fps                  | 1210        |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    to

Wrap the lander so it hops to a new x-position each episode (pad still at 0), log fuel and landing error, and then let PPO train for 1 million timesteps on that variant. At the end I save both the model weights and the normalization stats.

In [ ]:
# Cell 2: Playback to confirm random starts

# re-define wrappers from Cell 1:
class RandomStartWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        shift = self.np_random.uniform(-0.4, 0.4)
        uw = self.unwrapped
        uw.lander.position = (uw.lander.position.x + shift, uw.lander.position.y)
        obs = obs.copy(); obs[0] += shift
        return obs, info
    def step(self, action): return super().step(action)

class LunarWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs); return obs, info
    def step(self, action):
        obs, reward, term, trunc, info = super().step(action)
        info["fuel_used"] = 1.0 if action==2 else 0.0
        return obs, reward, term, trunc, info

def play_back(model_path, n_steps=500, sleep=0.016):
    def make_env():
        e = gym.make("LunarLander-v3", render_mode="human")
        e = RandomStartWrapper(e)
        e = LunarWrapper(e)
        return e

    ev = DummyVecEnv([make_env])
    ev = VecMonitor(ev)
    ev = VecNormalize.load("ppo_vecnormalize.pkl", ev)
    ev.training=False; ev.norm_reward=False

    m = PPO.load(model_path, env=ev)
    obs = ev.reset()
    for i in range(n_steps):
        action, i = m.predict(obs, deterministic=True)
        obs, i, dones, i = ev.step(action)
        ev.render()
        time.sleep(sleep)
        if dones[0]: break
    ev.close()

print(" Playback random-start policy:")
play_back("ppo_lunar_randstart.zip")


 Playback random-start policy:


Reload the saved policy plus the same normalization, then step through up to 500 frames in human‐render mode so you can actually see how the agent handles the random starts in real time.

In [ ]:
# Cell 3: Quantitative evaluation on random-start 

# re-define wrappers:
class RandomStartWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        shift = self.np_random.uniform(-0.4, 0.4)
        uw = self.unwrapped
        uw.lander.position = (uw.lander.position.x + shift, uw.lander.position.y)
        obs = obs.copy(); obs[0] += shift
        return obs, info
    def step(self, action): return super().step(action)

class LunarWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs); return obs, info
    def step(self, action):
        obs, reward, term, trunc, info = super().step(action)
        info["fuel_used"] = 1.0 if action==2 else 0.0
        return obs, reward, term, trunc, info

def make_eval_env():
    e = gym.make("LunarLander-v3")
    e = RandomStartWrapper(e)
    e = LunarWrapper(e)
    return e

ev = DummyVecEnv([make_eval_env])
ev = VecMonitor(ev)
ev = VecNormalize.load("ppo_vecnormalize.pkl", ev)
ev.training=False; ev.norm_reward=False

m = PPO.load("ppo_lunar_randstart.zip", env=ev)

def evaluate(model, env, n_episodes=50):
    rewards, fuels = [], []
    for i in range(n_episodes):
        obs = env.reset(); done=False; r_total=0; f_total=0
        while not done:
            action, i = model.predict(obs, deterministic=True)
            obs, reward, dones, infos = env.step(action)
            r_total += reward[0]
            f_total += infos[0]["fuel_used"]
            done = dones[0]
        rewards.append(r_total); fuels.append(f_total)
    print(f"Reward: {mean(rewards):.1f} ± {stdev(rewards):.1f}")
    print(f"Fuel:   {mean(fuels):.1f} ± {stdev(fuels):.1f}")

evaluate(m, ev, n_episodes=50)
ev.close()


Reward: 220.9 ± 54.3
Fuel:   153.0 ± 24.8


Run 50 test episodes with the random‐start wrapper (no rendering), record reward and total fuel each time, and print out the mean ± std so you can see the policy’s average performance and its variability.